# Integracion Continua

Fuente: <a href="https://www.youtube.com/watch?v=ILXR7ScxEHI">Video</a>

- Es una herramienta que nos ayuda a combatir el Merge Hell
- La integracion continua es hacer la mayor cantidad de sincronizaciones posibles en el tiempo.

## Github Actions

Github ofrece un servicio para hacer la integracion continua utilizando el plugin: tox-gh-actions . Esto basicamente nos va permitir ejecutar tox con cada push/pull. Este servicio se ejecuta en los servidores de github.

La idea basicamente es, cada vez que haga una actualizacion correr tox. Entonces podemos ver si hay algun conflicto.

El integrador continuo que vamos a ver es el de github, pero existen otros, como el de gitlab y uno que es exclusivamente para realizar integracion continua que es travis.yml. Sin embargo este cuenta dinero.

Se recomienda tener mas de un integrador continuo.

## Pasos para realizar la integracion continua con Github Actions

1. Configurar el archivo tox.ini agregando una seccion para gh-actions :

    - [gh-actions] 12:00
    - 

2. Agregar un archivo .yml
    - Debe estar colocadoen este path: ProyectoX/.github/workflow/ci.yml
    - .github/workflow es mandatorio
    - El archivo de integracion ci.yml puede tener cualquier nombre en realidad.

Cuando hagamos un pull/push - si vamos al apartado de Acions en GITHUB, veremos como los entornos se van ejecutando.

- Si se falla se vera una cruz y se enviara un email a la persona que hizo el push.

- cuando se hace pull/push se ejecuta tox, pero esto no impide que ese pull/push se haga. Asi que ojo con eso.

In [ ]:
# Contenido de ci.yml

name: ProyectoX     <--- Nombre del proyecto

on:                <---- Esto indica que el archivo workflow se va a ejecutar en cada una de estas acciones de git.
  - push
  - pull_request

jobs:                     <------ Esta seccion describe lo que estaremos ejecutando. A partir de aca es copiar la configuracion.
  build:

    runs-on: ubuntu-latest    <--- Sistema Operativo
    strategy:           <---- Matriz de estrategia
      matrix:
        python-version: [3.8, 3.9]              <---- Incluir todas las versiones de python que se quieran probar.

    steps:                                      <----- Pasos a Ejecutar.
      - uses: actions/checkout@master
      - name: Set up Python ${{ matrix.python-version }}   <--- Hace referencia a matrix>python-version de arriba.
        uses: actions/setup-python@v2       <--- v2 tiene la version de python 3.9
        with:
          python-version: ${{ matrix.python-version}}
      - name: Install tox   <-------- Nombre descriptivo de lo que se esta ejecutando
        run: pip install tox tox-gh-actions  <------ Instala tox y el pluggin de github actions.
      - name: Test with tox
        run: tox -r     <---- Regenarar los entornos cada vez que se ejecute.
      - name: Test coverage
        run: pip install coveralls
        coveralls --service=github

## Version alternativa del proyecto Spyctral

In [ ]:
# ==============================================================
# CI with GitHub Actions
# ==============================================================

name: Tests

on:
  - push
  - pull_request

env:
  default-python: '3.10'

jobs:
  build:
    strategy:
      fail-fast: false
      matrix:
        os: [ubuntu-latest, macos-latest]
        tox_env: [py39, py310, py311, py312, py13]
        include:
          - tox_env: style
          - tox_env: docstyle
          - tox_env: check-manifest
          - tox_env: check-testdir
          - tox_env: check-headers
          - tox_end: check-apidocsdir
          - tox_env: coverage
            pandoc: true

     # The operative sistem
    runs-on: ${{ matrix.os || 'ubuntu-latest' }}

    # the steps of the CI
    steps:
      - uses: actions/checkout@master

      # optionals
      - name: Install pandoc
        if: ${{ matrix.pandoc }}
        run: sudo apt-get install pandoc

      - name: Determine Python version
        id: pyversion
        if: ${{ startsWith( matrix.tox_env, 'py3' ) }}
        uses: bluwy/substitute-string-action@v1
        with:
          _input-text: ${{ matrix.tox_env }}
          py3: "3."

      # tox steps
      - name: Set up Python ${{ steps.pyversion.outputs.result || env.default-python }}
        uses: actions/setup-python@v2
        with:
          python-version: ${{ steps.pyversion.outputs.result || env.default-python }}

      - name: Install tox
        run: pip install tox

      - name: tox -re ${{ matrix.tox_env }}
        run: tox -re ${{ matrix.tox_env }}


## Un poco mas a fondo con github actions:

Fuente: Excelente <a href="https://www.youtube.com/watch?v=R8_veQiYBjI">Video</a>

- Github actions es una plataforma para automatizar workflows. Cuando decimos esto, CI = Continuous Integration y CD = Continuous Deplyment, es uno de muchos workflows que vamos a poder encontrar.

- Cuales serian estos workflows mas generalmente? Son basicamente distintas situaciones, que queremos automatizar lo mas posible. Por ejemplo, acciones que debemos tomar cuando hay algun issue con el repositorio.

- Github actions nos permite automatizar estos workflows. Siempre que suceda algo en nuesro repositorio se ejecutaran respuestas automaticamente en respuesta. Basicamente de eso se trata

- Que significa que algo suceda en nuestro repositorio?. Concretamente estas 'cosas' que suceden, se denominan eventos en este ecosistema. Estos pueden ser:
    - pull request
    - issue
    - merging pull in master
    - person joining
    - aplication and tools

- La manera en que funciona esto es simple.
    - Se escucha por un evento
    - Se responde a ese evento con un trigger de workflow


## CI/CD con github actions

Uno de los flujos de trabajo mas comunes en un repositorio es el siguiente:

Commit code -> Test -> Build -> Push -> Deploy

En teoria una de las ventajas mas grandes de github actions es que es una herramienta facil y que esta pensada para desarrolladores. Por lo cual no requiere que haya una persona especializada para ejecutarla.

En el video se muestra como para realizar un proyecto necesitariamos muchas tecnologias. Las cuales , si las quisieramos probar en una computadora , necesitariamos re instalar un monton de servicios y aplicaciones. Github actions nos provee ya de un entorno con todas las tecnologias y a plicaciones que necesitemos.

### Crear un template para nuestro proyecto

Si vamos a nuestro repositorio y vamos al apartado de actions, veremos que se nos ofrecen varios templates sugeridos para el repositorio. Podemos empezar entonces desde aqui.

## Sintaxys

<a href="https://docs.github.com/en/actions/writing-workflows/choosing-when-your-workflow-runs/events-that-trigger-workflows">Eventos</a> que triggerean un workflow (doc oficial)

### steps
- steps [requerido] : Usado para: run commands - setup tasks - run action
    - uses : select actions
    - run : runs a command line command
    - with :  allows you to pass inputs to an action to configure how that action will run. The with block contains key-value pairs where the key is the input parameter, and the value is the setting or argument you want to provide to the action.

### jobs
- jobs : Cada job en un workflow corre sobre un entorno separado. Podemos argumentar que hay entonces paralelismo. La notacion de esta es basicamente asi:

Elementos:

- El nombre (solamente se escribe)
- runs-on (especifica el sistema operativo)
- strategy: In GitHub Actions, the strategy keyword is used within a job to define how a job should be executed across multiple configurations, such as different operating systems, Python versions, or other variables. 






- jobs
    - runs-on (especifica el sistema operativo)
    - < nombre del job 1>
    - etc

    - < nombre del job 2>
    - runs-on (especifica el sistema operativo)
    - etc

Como los jobs se ejecutan en paralelo, si lo que se necesita es que se corran en serie se puede utilizar la palabra needs.

- jobs
    - < nombre del job 1>
    - runs-on (especifica el sistema operativo)
    - etc

    - < nombre del job 2>
    - runs-on (especifica el sistema operativo)
    - needs: < nombre del job 1>
    - etc

### Notas:
- When you use actions/checkout@v2 (or any version of actions/checkout), it checks out the code based on the branch that triggered the workflow.
- Sobre las versiones de python: Esto puede producir muchos errores, si uno escribe por ejemplo versiones mayores a 3.10 estas se toman como 3.1. Por lo cual las notaciones permitidas serian:
    - [py39, py310, py311, py312, py13]
    - ['3.9', '3.10', '3.11', '3.12', '3.13']